# CVX Tutorial

Work on `cvxpy` from the site's [tutorial](https://www.cvxpy.org/tutorial/index.html) and [examples](https://www.cvxpy.org/examples/), respectively.

In [70]:
# Libraries
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

# Part 1: Tutorial
## 1. What is CVXPY?
### 1.1 Changing the problem
__Problem__ : immutable!
$\to$ to change objective/constraints, create new problem

For example, consider the simple optimization problem:

In [71]:
# Two scalar optimization variables
x = cp.Variable()
y = cp.Variable()

# Two constraints
constraints = [x + y == 1,
               x - y >= 1]

# Objective
objective = cp.Minimize((x - y)**2)

# Form & solve problem
prob = cp.Problem(objective, constraints)
prob.solve()  # returns optimal value
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", x.value, y.value)

status: optimal
optimal value 1.0
optimal var 1.0 1.570086213240983e-22


Now, create new problem

In [72]:
# Replace the objective!
objective2 = cp.Maximize(x + y)
prob2 = cp.Problem(objective2, prob.constraints)
print("optimal value", prob2.solve())

# Replace constraint (x + y == 1)
constraints = [x + y <= 3] + prob2.constraints[1:]
prob3 = cp.Problem(prob2.objective, constraints)
print("optimal value", prob3.solve())

optimal value 0.9999999999945574
optimal value 2.999999999974675


### 1.2 Infeasible and unbounded problems

If infeasible/unbounded:
- status reflects this
- value fields not updated

_Optimal value_ for minimization (opposite for max.):
- `inf` if infeasible
- `-inf` if unbounded

In [73]:
# Univariate scalar
x = cp.Variable()

# Infeasible problem
objective = cp.Minimize(x)
constraints = [x >= 1, x <= 0]
prob = cp.Problem(objective, constraints)
print("status:", prob.status)
print("optimal value", prob.value)

status: None
optimal value None


In [74]:
# Unbounded problem
prob = cp.Problem(cp.Minimize(x))
prob.solve()
print("status:", prob.status)
print("optimal value", prob.value)

status: unbounded
optimal value -inf


### 1.3 Other problem statuses

Aliases for status strings:
- If problem solved, but to lower-than-desired accuracy, then one of:
    - `OPTIMAL`
    - `INFEASIBLE`
    - `UNBOUNDED`
- Otherwise:
    - `OPTIMAL_INACCURATE`
    - `INFEASIBLE_INACCURATE`
    - `UNBOUNDED INACCURATE`
    
If completely fails to solve, then `SolverError` (choose another solver?)

To test if problem solved successfully:
```
prob.status == OPTIMAL
```

### 1.4 Vectors and matrices

In [75]:
# Scalar
a = cp.Variable()

# Vector of shape (5, )
x = cp.Variable(5)

# Matrix of shape (4, 7)
x = cp.Variable((4, 7))

_Example_ : see least-squares with box constraint!

### 1.5 Constraints
_Can_ use:
- `==`, `<=`, and `>=` (always elementwise)

__CANNOT USE__:
- `>` and `<`
- Chained constraints (e.g. `0 <= x <= 1` or `x == y == 2`)

### 1.6 Parameters
__Parameter__ : symbolic constant
- Allow changing of value of constant w/o reconstructing entire problem
- Can specify attributes -- see:
    - Disciplined Parameterized Programming (DPP)
    - Disciplined Convex Programming (DCP)
- Can be assigned const value any time after creation

_Examples_ :

In [76]:
# Positive scalar parameter
m = cp.Parameter(nonneg = True)

# Col vector param w/unk sign (by default)
c = cp.Parameter(5)

# Matrix param w/negative entries
G = cp.Parameter((4, 7), nonpos = True)

# Assigns const value to G
G.value = -np.ones((4, 7))

In [77]:
G

Parameter((4, 7), nonpos=True)

In [78]:
G.value

array([[-1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1.]])

Can also init param w/value:

In [79]:
# Create param, then assign
rho = cp.Parameter(nonneg = True)
rho.value = 2

# Equivalent init param w/value
rho = cp.Parameter(nonneg = True, value = 2)

Parameters commonly used to compute __trade-off curves__, as in LASSO (code doesn't work?)

## 2. Disciplined convex programming

### 2.1 Expressions
_Examples_ :

In [102]:
# Create variables and parameters.
x, y = cp.Variable(), cp.Variable()
a, b = cp.Parameter(), cp.Parameter()

# Examples of CVXPY expressions.
3.69 + b/3
x - 4*a
cp.sqrt(x) - cp.minimum(y, x - a)
cp.maximum(2.66 - cp.sqrt(y), cp.square(x + 2*y))

Expression(CONVEX, NONNEGATIVE, ())

- Dimensions: `expr.shape`
- Total entries: `expr.size`
- Number of dimensions: `expr.ndim`

In [105]:
X = cp.Variable((5, 4))
A = numpy.ones((3, 5))

# Use expr.shape to get the dimensions.
print("dimensions of X:", X.shape)
print("size of X:", X.size)
print("number of dimensions:", X.ndim)
print("dimensions of sum(X):", cp.sum(X).shape)
print("dimensions of A @ X:", (A @ X).shape)

# ValueError raised for invalid dimensions.
try:
    A + X
except ValueError as e:
    print(e)

dimensions of X: (5, 4)
size of X: 20
number of dimensions: 2
dimensions of sum(X): ()
dimensions of A @ X: (3, 4)
Cannot broadcast dimensions  (3, 5) (5, 4)


### 2.2 Sign
Each (sub)expression flagged as
- _positive_ (non-negative) -- if even # of same signs
- _negative_ (non-positive) -- if odd # of same, known signs
- _zero_ (if any expression has sign 0)
- _unknown_ (if either expression has unknown sign)

Sign in `expr.sign`:

In [106]:
x = cp.Variable()
a = cp.Parameter(nonpos=True)
c = numpy.array([1, -1])

print("sign of x:", x.sign)
print("sign of a:", a.sign)
print("sign of square(x):", cp.square(x).sign)
print("sign of c*a:", (c*a).sign)

sign of x: UNKNOWN
sign of a: NONPOSITIVE
sign of square(x): NONNEGATIVE
sign of c*a: UNKNOWN


### 2.3 Curvature

_Options_:
- constant
- affine (also holds for constant)
- convex
- concave
- unknown

Recall affine $\implies$ convex & concave

### 2.4 Curvature rules
Based on __general composition theorem__ to each (sub)expression:
1. $f(e_1, \, \dots, \, e_n)$ convex if $f$ convex and for each $e_i$, one of the following holds:
    - $f$ increasing in argument $i$ and $e_i$ is convex
    - $f$ decreasing in argument $i$ and $e_i$ is concave
    - $e_i$ is affine or constant
2. $f(e_1, \, \dots, \, e_n)$ convace if $f$ concave and for each $e_i$, one of the following holds:
    - $f$ increasing in argument $i$ and $e_i$ is concave
    - $f$ decreasing in argument $i$ and $e_i$ is convex
    - $e_i$ is affine or constant
3. $f(e_1, \, \dots, \, e_n)$ affine if $f$ affine and each $e_i$ affine
4. If none of the above apply, then unknown!

_Notes_ :
- Increasing depends on sign of argument
- Curvature in `expr.curvature`

In [107]:
x = cp.Variable()
a = cp.Parameter(nonneg=True)

print("curvature of x:", x.curvature)
print("curvature of a:", a.curvature)
print("curvature of square(x):", cp.square(x).curvature)
print("curvature of sqrt(x):", cp.sqrt(x).curvature)

curvature of x: AFFINE
curvature of a: CONSTANT
curvature of square(x): CONVEX
curvature of sqrt(x): CONCAVE


### 2.5 Infix operators

`+, -, *, /` and `@` __infix operators__ treated like functions -- see more on tutorial site...

### 2.6 DCP problems

Objective must have one of two forms:
- Minimize (convex)
- Maximize (concave_

Only valid constraints under DCP rules:
- affine == affine
- convex <= concave
- concave >= convex

_Can check_ if satisfies DCP by `object.is_dcp()

In [109]:
# Vars
x = cp.Variable()
y = cp.Variable()

# Some DCP problems
prob1 = cp.Problem(cp.Minimize(cp.square(x - y)),
                   [x + y >= 0])
prob2 = cp.Problem(cp.Maximize(cp.sqrt(x - y)),
                   [2*x - 3 == y,
                    cp.square(x) <= 2])

print("prob1 is DCP:", prob1.is_dcp())
print("prob2 is DCP:", prob2.is_dcp())

prob1 is DCP: True
prob2 is DCP: True


Some example non-DCP problems:

In [110]:
# Non-DCP objective
obj = cp.Maximize(cp.square(x))
prob3 = cp.Problem(obj)

print("prob3 is DCP:", prob3.is_dcp())
print("Maximize(square(x)) is DCP:", obj.is_dcp())

# Non-DCP constraint
prob4 = cp.Problem(cp.Minimize(cp.square(x)),
                   [cp.sqrt(x) <= 2])

print("prob4 is DCP:", prob4.is_dcp())
print("sqrt(x) <= 2 is DCP:", (cp.sqrt(x) <= 2).is_dcp())

prob3 is DCP: False
Maximize(square(x)) is DCP: False
prob4 is DCP: False
sqrt(x) <= 2 is DCP: False


Error raised if `problem.solve()` on non-DCP problem!

In [111]:
# Non-DCP problem
prob = cp.Problem(cp.Minimize(cp.sqrt(x)))

try:
    prob.solve()
except Exception as e:
    print(e)

Problem does not follow DCP rules. Specifically:
The objective is not DCP, even though each sub-expression is.
You are trying to minimize a function that is concave.
However, the problem does follow DQCP rules. Consider calling solve() with `qcp=True`.


# Part 2: Examples
## 1. Least Squares
### 1.1 Vanilla
$$\text{minimize} \;\; \|A x - b\|_2^2$$

In [80]:
# Data
m = 20
n = 15
np.random.seed(1)
A = np.random.randn(m, n)
b = np.random.randn(m)

# Define the problem
x = cp.Variable(n)
cost = cp.sum_squares(A @ x - b)
prob = cp.Problem(cp.Minimize(cost))
prob.solve()

7.005909828287484

In [81]:
# Print the result
print("Optimal value: ", prob.value)
print("Optimal x: ")
print(x.value)
print("Norm of residual: ", cp.norm(A @ x - b, p = 2).value)

Optimal value:  7.005909828287484
Optimal x: 
[ 0.17492418 -0.38102551  0.34732251  0.0173098  -0.0845784  -0.08134019
  0.293119    0.27019762  0.17493179 -0.23953449  0.64097935 -0.41633637
  0.12799688  0.1063942  -0.32158411]
Norm of residual:  2.6468679280023557


#### 1.2 Least squares with box constraint

$$
\begin{align}
&\text{minimize} \;\; \|A x - b\|_2^2 \\
&\text{subject to} \;\; x \in [0, 1]
\end{align}
$$

In [82]:
# Data
m = 30
n = 20
np.random.seed(1)
A = np.random.randn(m, n)
b = np.random.randn(m)

# Problem construction
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A @ x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

Optimal objective value in `prob.solve()`

In [83]:
result = prob.solve()
result

19.83126370644502

Optimal value for `x` in `x.value`

In [84]:
print(x.value)

[-1.79109255e-19  2.85112420e-02  2.79973443e-19  3.37658729e-20
 -2.72802663e-19  1.49285011e-01 -9.94082533e-20  8.35373900e-20
  2.46718649e-01  5.78224144e-01 -4.03739463e-19  1.01242860e-03
 -9.28486180e-20  2.26767464e-01 -1.58813678e-19 -8.97232272e-20
 -1.22145729e-19 -1.51509428e-19  1.12060672e-19 -3.48318635e-19]


Optimal Lagrange multiplier for constraint in `constraint.dual_value`

In [85]:
print(constraints[0].dual_value)

[ 2.50938945  0.          2.78354615  1.79425782 13.08579183  0.
  0.73716363  3.35344995  0.          0.          8.93825054  0.
  7.02955161  0.          4.71068649  3.18873635  2.06090107 10.08166738
  3.0481157   8.53268239]


## 2. Linear program (LP)

__Problem statement__ :
$$
\begin{align}
&\text{minimize} \;\; c^T x \\
&\text{subject to} \;\; A x \leq b
\end{align}
$$

In [86]:
# Generate random non-trivial linear program
m = 15
n = 10
np.random.seed(1)
s0 = np.random.randn(m)
lamb0 = np.maximum(-s0, 0)
s0 = np.maximum(s0, 0)
x0 = np.random.randn(n)
A = np.random.randn(m, n)
b = A @ x0 + s0
c = -A.T @ lamb0

# Define & solve problem
x = cp.Variable(n)
objective = cp.Minimize(c.T @ x)
constraints = [A @ x <= b]
prob = cp.Problem(objective, constraints)
prob.solve()

-15.220912605552911

In [87]:
# Print results
print("The optimal value is", prob.value)
print("A solution x is")
print(x.value)
print("A dual solution is")
print(prob.constraints[0].dual_value)

The optimal value is -15.220912605552911
A solution x is
[-1.10133381 -0.16360111 -0.89734939  0.03216603  0.6069123  -1.12687348
  1.12967856  0.88176638  0.49075229  0.8984822 ]
A dual solution is
[6.98804542e-10 6.11756416e-01 5.28171747e-01 1.07296862e+00
 3.93758851e-09 2.30153870e+00 4.25703999e-10 7.61206896e-01
 8.36905532e-09 2.49370377e-01 1.30187000e-09 2.06014070e+00
 3.22417207e-01 3.84054343e-01 1.59493644e-09]


## 3. Quadratic program (QP)

### 3.1 Standard form

$$
\begin{align}
\text{minimize} \;\;\; &(1/2) x^T P x + q^T x \\
\text{subject to} \;\;\; &G x \preceq h \\
& A x = b
\end{align}
$$

### 3.2 Example: finance QP

$$
\begin{align}
\text{minimize} \;\;\;& (1/2) x^T \Sigma x - r^T x \\
\text{subject to} \;\;\;& x \preceq 0 \\
& \mathbf{1}^T x = 1
\end{align}
$$

Dual solution $\lambda^*$ such that $\lambda_i^* > 0 \implies g_i^T x \leq h_i$ holds with equality at $x^*$

In [88]:
# Generate data
m = 15
n = 10
p = 5
np.random.seed(1)
P = np.random.randn(n, n)
P = P.T @ P
q = np.random.randn(n)
G = np.random.randn(m, n)
h = G @ np.random.randn(n)
A = np.random.randn(p, n)
b = np.random.randn(p)

# Problem
x = cp.Variable(n)
objective = cp.Minimize((1/2)*cp.quad_form(x, P) + q.T @ x)
constraints = [G @ x <= h, 
               A @ x == b]
prob = cp.Problem(objective, constraints)

In [89]:
# Print result
prob.solve()
print("The optimal value is", prob.value)
print("A solution x is")
print(x.value)
print("A dual solution corresponding to the inequality constraints is")
print(prob.constraints[0].dual_value)

The optimal value is 86.89141585569918
A solution x is
[-1.68244521  0.29769913 -2.38772183 -2.79986015  1.18270433 -0.20911897
 -4.50993526  3.76683701 -0.45770675 -3.78589638]
A dual solution corresponding to the inequality constraints is
[ 0.          0.          0.          0.          0.         10.45538054
  0.          0.          0.         39.67365045  0.          0.
  0.         20.79927156  6.54115873]


## 4. Second-order cone program (SOCP)
### 4.1 General form

$$
\begin{align}
\text{minimize} \;\;\;& f^T x \\
\text{subject to} \;\;\;& \|A_i x + b_i\|_2 \leq c_i^T x + d_i, \;\;\; i = 1, \, \dots, \, m \\
& F x = g
\end{align}
$$

### 4.2 Example: robust linear program
__Original form__:
$$
\begin{align}
\text{minimize} \;\;\;& c^T x \\
\text{subject to} \;\;\; & (a_i + u_i)^T x \leq b_i \text{ for all } \|u_i\|_2 \leq 1, \;\;\; i = 1, \, \dots, \, m
\end{align}
$$

__SOCP form__:
$$
\begin{align}
\text{minimize} \;\;\;& c^T x \\
\text{subject to} \;\;\; & a_i^T x + \|x\|_2 \leq b_i, \;\;\; i = 1, \, \dots, \, m
\end{align}
$$

In [90]:
# Parameters
m = 3
n = 10
p = 5
n_i = 5
np.random.seed(2)
f = np.random.randn(n)
A = []
b = []
c = []
d = []
x0 = np.random.randn(n)

# Generate data
for i in range(m):
    A.append(np.random.randn(n_i, n))
    b.append(np.random.randn(n_i))
    c.append(np.random.randn(n))
    d.append(np.linalg.norm(A[i] @ x0 + b, 2) - c[i].T @ x0)
F = np.random.randn(p, n)
g = F @ x0

Use `cp.SOC(t, x)` to create SOC constraint $\|x\|_2 \leq t$

In [91]:
# Define problem
x = cp.Variable(n)
soc_constraints = [
    cp.SOC(c[i].T @ x + d[i], A[i] @ x + b[i]) for i in range(m)
]
objective = cp.Minimize(f.T @ x)
constraints = soc_constraints + [F @ x == g]
prob = cp.Problem(objective, constraints)

In [92]:
# Solve problem
prob.solve()
print("The optimal value is", prob.value)
print("A solution x is")
print(x.value)
for i in range(m):
    print("SOC constraint %i dual variable solution" % i)
    print(soc_constraints[i].dual_value)

The optimal value is -9.582695716265503
A solution x is
[ 1.40303325  2.4194569   1.69146656 -0.26922215  1.30825472 -0.70834842
  0.19313706  1.64153496  0.47698583  0.66581033]
SOC constraint 0 dual variable solution
[array([0.61662526]), array([[ 0.35370661],
       [-0.02327185],
       [ 0.04253095],
       [ 0.06243588],
       [ 0.49886837]])]
SOC constraint 1 dual variable solution
[array([0.35283078]), array([[-0.14301082],
       [ 0.16539699],
       [-0.22027817],
       [ 0.15440264],
       [ 0.06571645]])]
SOC constraint 2 dual variable solution
[array([0.86510445]), array([[-0.114638  ],
       [-0.449291  ],
       [ 0.37810251],
       [-0.6144058 ],
       [-0.11377797]])]
